In [7]:
!pip install -q google-adk


[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [8]:
import os
try:
    from google.colab import userdata
    os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
except ImportError:
    pass  # Not running in Colab; uses env var already set

# Website Builder Agent

Build an agent that generates websites from descriptions. Your agent will have these tools:

1. **generate_page** — Generate a complete HTML page for a given page type and description
2. **add_section** — Add a section (hero, features, testimonials, contact) to a page
3. **apply_theme** — Apply a visual theme (modern, minimal, bold) to HTML with CSS

Tools return hardcoded/fake data — the goal is designing the **interface** between the model and your code.

## Setup

In [9]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.genai import types

## TODO 1: Define your tools

Just write plain Python functions. The ADK infers the schema from:
- **Type hints** → parameter types
- **Docstring** → tool description

The first tool is done for you as an example. Implement the remaining two.

In [10]:
# Example tool (complete)

def generate_page(page_type: str, description: str) -> dict:
    """Generate a complete HTML page based on a page type and description.

    Args:
        page_type: The type of page to generate (e.g. "landing", "portfolio", "blog").
        description: A description of what the page should contain.

    Returns:
        dict: status and the generated HTML or error message.
    """
    html = (
        "<!DOCTYPE html>\n"
        "<html lang=\"en\">\n"
        "<head>\n"
        "  <meta charset=\"UTF-8\">\n"
        "  <meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\">\n"
        f"  <title>{page_type.title()} Page</title>\n"
        "  <style>\n"
        "    * { margin: 0; padding: 0; box-sizing: border-box; }\n"
        "    body { font-family: Arial, sans-serif; line-height: 1.6; color: #333; }\n"
        "    .container { max-width: 1200px; margin: 0 auto; padding: 0 20px; }\n"
        "    header { background: #2c3e50; color: white; padding: 1rem 0; }\n"
        "    main { padding: 2rem 0; }\n"
        "    footer { background: #34495e; color: white; padding: 1rem 0; text-align: center; }\n"
        "  </style>\n"
        "</head>\n"
        "<body>\n"
        "  <header><div class=\"container\"><h1>" + page_type.title() + " Page</h1></div></header>\n"
        "  <main><div class=\"container\">\n"
        f"    <p>{description}</p>\n"
        "  </div></main>\n"
        "  <footer><div class=\"container\"><p>&copy; 2025</p></div></footer>\n"
        "</body>\n"
        "</html>"
    )
    return {
        "status": "success",
        "report": f"Generated a {page_type} page. Here is the HTML:\n\n{html}",
    }


def add_section(section_type: str, content: str) -> dict:
    """Generate an HTML section to add to a page.

    Args:
        section_type: The type of section (e.g. "hero", "features", "testimonials", "contact").
        content: The text content for the section.

    Returns:
        dict: status and the generated HTML section or error message.
    """
    # TODO: implement — return a dict with "status" and "report" keys
    # Hint: return a hardcoded HTML snippet for the section_type, incorporating content
    # Example: a "hero" section might be a <section> with a heading, paragraph, and a CTA button
    pass


def apply_theme(html: str, theme: str) -> dict:
    """Apply a visual theme to an HTML page by injecting CSS styles.

    Args:
        html: The HTML string to apply the theme to.
        theme: The theme name (e.g. "modern", "minimal", "bold").

    Returns:
        dict: status and the themed HTML or error message.
    """
    # TODO: implement — return a dict with "status" and "report" keys
    # Hint: define a CSS string for each theme and inject it into the <head> of the HTML
    # Example: "modern" might use a gradient background and rounded corners
    pass

## TODO 2: Define your agent

In [11]:
agent = Agent(
    model="gemini-3-flash-preview",
    name="website_builder",
    # TODO: Write an instruction that gives the agent a web-developer persona.
    # Hint: describe what the agent can help with, its tone, and any guidelines.
    instruction="""You are a helpful website builder assistant.

Replace this with your agent's persona and guidelines.
""",
    tools=[generate_page, add_section, apply_theme],
)

## Run your agent

Once you've filled in the TODOs above, run this cell to test your agent!

In [13]:
runner = InMemoryRunner(agent=agent, app_name="website_builder")
runner.auto_create_session = True
user_content = types.Content(role="user", parts=[types.Part.from_text(text="Create a landing page like Harvey")])

final_text = ""
async for event in runner.run_async(user_id="user1", session_id="session1", new_message=user_content):
    if event.content and event.content.parts:
        for part in event.content.parts:
            if part.function_call:
                print(f"🔧 Tool call: {part.function_call.name}({dict(part.function_call.args)})")
            if part.function_response:
                print(f"  ↳ Response: {part.function_response.name} returned")
            if part.text:
                final_text = part.text

print(f"\n{'='*60}\nFinal response:\n{'='*60}\n{final_text}")

🔧 Tool call: generate_page({'page_type': 'landing', 'description': "A sophisticated, minimalist landing page for 'LexisAI', an enterprise AI platform for legal and professional services. The page should feature a dark theme, high-end typography, and a hero section with the headline 'The Operating System for Professional Services'. Include a 'Request Demo' primary call to action and a 'Sign In' secondary action. The design should feel secure, professional, and cutting-edge."})
  ↳ Response: generate_page returned
🔧 Tool call: add_section({'content': "LexisAI: The Operating System for Professional Services. Empowering the world's leading firms with secure, enterprise-grade AI. LexisAI automates complex workflows across legal, tax, and advisory services, allowing professionals to focus on higher-value work. [Request Demo] [Sign In]", 'section_type': 'hero'})
  ↳ Response: add_section returned
🔧 Tool call: add_section({'content': 'Advanced Research: Instantly search across vast repositorie

## Tips

- **Docstrings matter** — the ADK uses them as tool descriptions. Be specific about what each parameter means.
- **Fake data is fine** — the goal is designing the agent's interface, not building a real website generator.
- **Test edge cases** — what happens when a user asks for a page type your tools don't handle?

### Stretch Goals
- Add a `generate_navbar` tool that creates navigation bars with custom links
- Support dark mode in your `apply_theme` tool
- Handle multi-page sites (e.g. "create a homepage and an about page")